In [2]:
import numpy as np
import pandas as pd
import model
import nessra_genes_sets_generator as ng
import mygene 
# Import train_test_split function
from sklearn.model_selection import train_test_split
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Import `StandardScaler` from `sklearn.preprocessing`
from sklearn.preprocessing import StandardScaler
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB

In [25]:
#------------ SVM ANALYSIS ------------
def svmModel(matrix_t, list_of_sets, obj):
    mat = model.select_table(matrix_t, list_of_sets)
    print("Transpose matrix filtered shape: {}".format(mat.shape))
    
    # Split dataset into training set and test set
    X_train, X_test, y_train, y_test = train_test_split(mat.values, obj.values, test_size=0.3,random_state=100) # 70% training and 30% test
    # Define the scaler 
    scaler = StandardScaler().fit(X_train)
    # Scale the train set
    X_train = scaler.transform(X_train)
    # Scale the test set
    X_test = scaler.transform(X_test)
    
    #model training
    clf_ns = model.training(X_train, y_train)

    y_pred = clf_ns.predict(X_test)
    print("")
    print("---- SVM ----")
    print("Proportion of 0s in test y: {}".format(len([x for x in y_test if x <1])/len(y_test)))
    print("Proportion of 0s in prediction y: {}".format(len([x for x in y_pred if x <1])/len(y_pred)))
    print("")
    print(y_test)
    print(y_pred)
    print("AUC:", metrics.roc_auc_score(y_test, y_pred))
    
    return(clf_ns)

def BayesModel(matrix_t, list_of_sets, obj):
    mat = model.select_table(matrix_t, list_of_sets)
    print("Transpose matrix filtered shape: {}".format(mat.shape))
    
    # Split dataset into training set and test set
    X_train, X_test, y_train, y_test = train_test_split(mat.values, obj.values, test_size=0.3,random_state=100) # 70% training and 30% test
    # Define the scaler 
    scaler = StandardScaler().fit(X_train)
    # Scale the train set
    X_train = scaler.transform(X_train)
    # Scale the test set
    X_test = scaler.transform(X_test)

    #Create a Gaussian Classifier
    g_model = GaussianNB()

    # Train the model using the training sets
    g_model.fit(X_train,y_train)

    predicted= g_model.predict(X_test)

    print("---- Naive Bayes ----")
    print("Proportion of 0s in test y: {}".format(round(len([x for x in y_test if x <1])/len(y_test), 4)))
    print("Proportion of 0s in prediction y: {}".format(round(len([x for x in predicted if x <1])/len(predicted),4)))
    print("")
    print(y_test)
    print(predicted)
    print("AUC:",round(metrics.roc_auc_score(y_test, predicted), 4))
    
    return g_model

# Matrix and Objective vector preparation

In [15]:
#------------- MATRIX and CLASS VECTOR
matrix = pd.read_csv("~/Desktop/expressionMatrix5.tsv", sep = "\t", header=0)
#class_vector = pd.read_csv("~/GoogleDrive/DataMiningLab/Model/objective.tsv", sep = "\t", header=0)
print("Matrix shape: {}".format(matrix.shape))
#print(matrix.head(2))
print("")

# transposing matrix
matrix_t = matrix.T
matrix_t.columns = matrix_t.iloc[0]
matrix_t = matrix_t.drop("Gene_ID", axis=0)
print("Transpose matrix shape: {}".format(matrix_t.shape))
print("")

# class vector 
class_vector = pd.read_csv("~/Desktop/objective.tsv", sep = "\t", header=0)
print("Objective shape: {}".format(class_vector.shape))  

# generating barcode of samples
samples_bar = []
for name in list(matrix_t.index.values):
    samples_bar.append(name[:-4])
fil_for_class = class_vector.barcode.isin(samples_bar)
class_vector = class_vector[fil_for_class]
print("Objective shape after filter for matrix samples: {}".format(class_vector.shape))

# Final vector for classifying
obj = class_vector["over2years"]
print("Final obj vector for classication is a {}, len: {}".format(type(obj), len(obj)))
print("")

Matrix shape: (60482, 103)

Transpose matrix shape: (102, 60482)

Objective shape: (104, 3)
Objective shape after filter for matrix samples: (102, 3)
Final obj vector for classication is a <class 'pandas.core.series.Series'>, len: 102



# CorEX genes analysis

In [41]:
corex_mis_path = "./corex_mis.tsv"
cluster_id = 0
n_genes = 1300

c_genes = model.select_genes(corex_mis_path, cluster_id, n_genes)
print("Len of selected genes: {}".format(len(c_genes)))

Len of selected genes: 1300


In [37]:
# SVM analysis

svm = svmModel(matrix_t, [set(c_genes)], obj)

Transpose matrix filtered shape: (102, 100)
{'C': 10.0, 'gamma': 1.0}
fit_time 0.0014259020487467449
score_time 0.005232810974121094
test_accuracy 0.6612318840579711
test_f1 0.0
test_precision 0.0
test_recall 0.0

---- SVM ----
Proportion of 0s in test y: 0.6451612903225806
Proportion of 0s in prediction y: 1.0

[0 0 0 1 1 0 0 0 0 1 1 1 0 0 1 0 0 0 0 0 0 0 1 1 0 1 1 0 0 1 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
AUC: 0.5


/home/riccardo/miniconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/home/riccardo/miniconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/riccardo/miniconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/riccardo/miniconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'pr

In [42]:
# Bayes

bayes = BayesModel(matrix_t, [set(c_genes)], obj)

Transpose matrix filtered shape: (102, 1300)
---- Naive Bayes ----
Proportion of 0s in test y: 0.6452
Proportion of 0s in prediction y: 0.4194

[0 0 0 1 1 0 0 0 0 1 1 1 0 0 1 0 0 0 0 0 0 0 1 1 0 1 1 0 0 1 0]
[1 0 0 1 1 1 1 0 0 1 0 1 1 0 1 0 0 1 0 0 1 1 1 1 1 1 0 0 1 1 0]
AUC: 0.6841
